In [ ]:
# Instalar la biblioteca Prophet
!pip install prophet

# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet  # Importar Prophet después de la instalación
import warnings

warnings.filterwarnings("ignore")  # Ignorar advertencias


In [ ]:
# Subir el archivo desde el sistema local
from google.colab import files
uploaded = files.upload()  # Abre un cuadro de diálogo para subir archivos

# Leer el archivo CSV en un DataFrame
df = pd.read_csv('Enso.csv')  # Asegúrate de que el nombre del archivo coincida


Saving Enso.csv to Enso (1).csv


In [ ]:
# Limpiar y corregir las fechas en el DataFrame
def clean_dates(df, date_column):
    """
    Esta función corrige las fechas inválidas en el DataFrame.
    """
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    # Remover filas con fechas inválidas
    df = df.dropna(subset=[date_column])
    return df

# Verificar y limpiar la columna de fecha si está presente
if 'Date' in df.columns:
    df = clean_dates(df, 'Date')
    df.set_index('Date', inplace=True)

# Verificar nuevamente las fechas después de la limpieza
print("Fechas después de la limpieza:")
print(df.index)


Fechas después de la limpieza:
DatetimeIndex(['1870-01-01', '1870-02-01', '1870-03-01', '1870-04-01',
               '1870-05-01', '1870-06-01', '1870-07-01', '1870-08-01',
               '1870-09-01', '1870-10-01',
               ...
               '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01',
               '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01',
               '2024-04-01', '2024-05-01'],
              dtype='datetime64[ns]', name='Date', length=1853, freq=None)


In [ ]:
# Definir una función para ajustar un modelo Prophet y devolver los estadísticos de predicción
def prophet_forecast(series, freq='D'):
    # Asegurarse de que la serie sea un vector unidimensional (sin nulos)
    series = series.dropna().astype(float)

    # Crear un DataFrame compatible con Prophet
    df_prophet = series.reset_index()
    df_prophet.columns = ['ds', 'y']  # Columnas requeridas por Prophet

    # Dividir los datos en conjunto de entrenamiento (70%) y prueba (30%)
    train_size = int(len(df_prophet) * 0.7)
    train, test = df_prophet[:train_size], df_prophet[train_size:]

    # Ajustar el modelo Prophet
    model = Prophet(daily_seasonality=True, yearly_seasonality=True)
    model.fit(train)

    # Crear un DataFrame para las fechas futuras
    future = model.make_future_dataframe(periods=len(test), freq=freq)

    # Generar pronósticos
    forecast = model.predict(future)

    # Extraer los valores pronosticados correspondientes al conjunto de prueba
    y_pred = forecast['yhat'].values[-len(test):]

    # Calcular métricas de evaluación
    mse = mean_squared_error(test['y'], y_pred)  # Error Cuadrático Medio
    rmse = np.sqrt(mse)  # Raíz del Error Cuadrático Medio
    mae = mean_absolute_error(test['y'], y_pred)  # Error Absoluto Medio
    r2 = r2_score(test['y'], y_pred)  # Coeficiente de Determinación

    # Calcular MAPE manejando valores cero
    mape = np.mean(np.abs((test['y'] - y_pred) / (test['y'] + np.finfo(float).eps))) * 100  # Evitar división por cero

    # Realizar la prueba de Dickey-Fuller aumentada para la estacionariedad
    adf_test = adfuller(series)
    adf_statistic = adf_test[0]  # Estadístico ADF
    adf_p_value = adf_test[1]  # Valor p de la prueba ADF

    # Devolver las métricas y los estadísticos en un diccionario
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'adf_statistic': adf_statistic,
        'adf_p_value': adf_p_value
    }

# Aplicar el modelo Prophet a cada una de las series de tiempo numéricas en el DataFrame
prophet_results = {}
for column in df.select_dtypes(include=[np.number]).columns:
    print(f"Procesando la serie de tiempo: {column}")
    prophet_results[column] = prophet_forecast(df[column])

# Crear un DataFrame para almacenar los resultados
results_df = pd.DataFrame(prophet_results).T  # Transponer para una mejor presentación
results_df.columns = ['MSE', 'RMSE', 'MAE', 'R²', 'MAPE', 'ADF Statistic', 'ADF P-value']  # Renombrar columnas

# Mostrar el DataFrame con los resultados
print("Resumen de los resultados de Prophet:")
print(results_df)

# Guardar el DataFrame en un archivo CSV
results_df.to_csv('prophet_model_comparison.csv', index=True)  # Guardar el DataFrame en un archivo CSV
files.download('prophet_model_comparison.csv')  # Descargar el archivo CSV generado




INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/ao2hclug.json


Procesando la serie de tiempo: nino12


DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/oqmbn1cm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=42484', 'data', 'file=/tmp/tmpagh989pv/ao2hclug.json', 'init=/tmp/tmpagh989pv/oqmbn1cm.json', 'output', 'file=/tmp/tmpagh989pv/prophet_model9hwgnqdp/prophet_model-20240802163847.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:38:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:38:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/_da86rmj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/9kiii75s.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBU

Procesando la serie de tiempo: nino3


16:38:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/i36jozoa.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpagh989pv/u6hem06x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92094', 'data', 'file=/tmp/tmpagh989pv/i36jozoa.json', 'init=/tmp/tmpagh989pv/u6hem06x.json', 'output', 'file=/tmp/tmpagh989pv/prophet_model4sewkh6g/prophet_model-20240802163849.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:38:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


...

Procesando la serie de tiempo: nino34


DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8299', 'data', 'file=/tmp/tmpagh989pv/fyph7oqj.json', 'init=/tmp/tmpagh989pv/kmw_ssjo.json', 'output', 'file=/tmp/tmpagh989pv/prophet_modelz8bj040g/prophet_model-20240802163849.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:38:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:38:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Resumen de los resultados de Prophet:
             MSE      RMSE       MAE        R²          MAPE  ADF Statistic  \
nino12  1.165270  1.079477  0.757610 -0.034389  6.156882e+14     -11.532023   
nino3   0.936240  0.967595  0.704455 -0.162008  1.112707e+15     -10.225140   
nino4   0.459242  0.677674  0.559611 -0.160522  2.532950e+14      -9.779188   
nino34  0.828655  0.910305  0.694851 -0.110776  5.946613e+14     -10.125987   

         ADF P-value  
nino12  3.811493e-21  
nino3   5.200632e-18  
nino4   6.805263e-17  
nino34  9.179166e-18  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>